In [113]:
import pandas as pd
import altair as alt
import plotly.express as px

### Data Import

In [114]:
filepath = "/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/clean/"
applications = pd.read_csv(filepath + "applications.csv", index_col=0)
adoptions = pd.read_csv(filepath + "adopter.csv", index_col=0)
calendar = pd.read_csv(filepath + "calendar.csv")


catdata = pd.read_csv(filepath + "catdata.csv", index_col=0)
hardtoplace = pd.read_csv(filepath + "hardtoplace.csv", index_col=0)

### Summary Statistics

In [115]:
applications.shape

(8251, 11)

In [116]:
applications.head()

,Application ID,Application Type,Application Received Date/Time,Status,Application Status Date/Time,Application Note,Person ID,City,State,County,Zip
0,4444701,Cat Adoption Application,"01/01/2021, 6:54 am",Accepted,"03/05/2022, 12:10 pm",—,MER-P-17076,Groveland,MA,MA,01834
1,4444702,Cat Adoption Application,"01/01/2021, 8:34 am",Accepted,"03/05/2022, 11:55 am",—,MER-P-17075,Lawrence,MA,Essex,01841
2,4444703,Cat Adoption Application,"01/01/2021, 11:22 am",Accepted,"03/05/2022, 11:54 am",—,MER-P-17074,Mattapan,MA,United states,02126
3,4444704,Cat Adoption Application,"01/01/2021, 3:27 pm",Accepted,"03/05/2022, 11:53 am",—,MER-P-17073,Lynnfield,MA,NaN,01940
4,4444705,Cat Adoption Application,"01/01/2021, 3:51 pm",Accepted,"03/05/2022, 11:51 am",—,MER-P-17072,Methuen,MA,NaN,01844


In [117]:
applications.columns

Index(['Application ID', 'Application Type', 'Application Received Date/Time',
       'Status', 'Application Status Date/Time', 'Application Note',
       'Person ID', 'City', 'State', 'County', 'Zip'],
      dtype='object')

In [118]:
applications["Year"] = pd.to_datetime(applications["Application Received Date/Time"], format='mixed').dt.year

In [119]:
applications["Year"].value_counts()

Year
2021    1892
2023    1844
2024    1747
2022    1700
2025    1068
Name: count, dtype: int64

In [136]:
applications["City and State"] = applications["City"] + ", " + applications["State"]

In [137]:
applications["City and State"].value_counts()

City and State
Haverhill, MA          590
Amesbury, MA           477
Newburyport, MA        380
Salisbury, MA          253
Boston, MA             175
                      ... 
Center Sandwich, NH      1
Warren, NH               1
Fresno, NH               1
Ashaway, RI              1
Georgetown, ME           1
Name: count, Length: 563, dtype: int64

In [121]:
adoptions.shape

(3053, 15)

In [122]:
adoptions.head()

,Date Of Adoption,Animal ID,Animal Name,Species,Primary Breed,Sex,Altered,Est. Birthdate,Microchip Number,By (User),Outcome Subtype,Adopter ID,City,State,Zip
0,01/03/2021,MER-A-158,Oberon,Cat,Domestic Medium Hair,Male,Yes,06/01/2015,985112007171202,mer_jmccormack,Adoption,MER-P-14391,Danvers,MA,01923
1,01/03/2021,MER-A-70,Cleopatra,Cat,Domestic Medium Hair,Female,Yes,11/05/2013,985141002756186,mer_jmccormack,Adoption,MER-P-14392,Hampton,NH,03842
2,01/06/2021,MER-A-130,Princess,Cat,Domestic Shorthair,Female,Yes,11/24/2013,985141002878981,mer_jmccormack,Adoption,MER-P-14394,Kingston,NH,03848
3,01/09/2021,MER-A-114,Mugshot,Cat,Domestic Shorthair,Male,Yes,10/06/2018,985112010662548,mer_jmccormack,Adoption,MER-P-14397,Ayer,MA,01432
4,01/12/2021,MER-A-159,Simon,Cat,Domestic Shorthair,Male,Yes,12/30/2012,985141002879119,mer_jmccormack,Adoption,MER-P-14428,Brighton,MA,02135


In [138]:
adoptions["City and State"] = adoptions["City"] + ", " + adoptions["State"]

In [139]:
adoptions["City and State"].value_counts()

City and State
Haverhill, MA           239
Amesbury, MA            224
Newburyport, MA         173
Salisbury, MA           110
Hampton, NH              67
                       ... 
Rockland, MA              1
Weare, NH                 1
Marshfield Hills, NH      1
South Portland, ME        1
Georgetown, ME            1
Name: count, Length: 284, dtype: int64

# Adoption Policy

## Q1: Are there patterns to adoption applicants that can be used for targeted marketing efforts in communities?

In [140]:
def alt_pie_chart(data, name_field, value_field, title):

    base = alt.Chart(
        data,
        title = title
    ).encode(
        alt.Theta(value_field).stack(True),
        alt.Color(name_field).scale(scheme="category20")
    )
    
    pie = base.mark_arc(
        outerRadius=150,
        opacity=0.9,
        stroke='black',
        strokeOpacity=0.5
    )

    text = base.mark_text(
        radius=165, size=15
    ).encode(
        text=value_field
    ).properties(
        height = 400
    )

    return (text + pie).configure_legend(
        orient="right",
        offset=250,
        symbolStrokeWidth=0
    )

In [141]:
city_counts = applications["City and State"].value_counts().reset_index()
city_counts = city_counts.rename(columns={"count":"Applications"})

In [142]:
city_counts.loc[city_counts['Applications'] < 10, 'City and State'] = 'Other'

In [144]:
alt_pie_chart(city_counts, "City and State", "Applications", "Test")

alt.LayerChart(...)

In [145]:
alt_pie_chart(city_counts[:20], "City and State", "Applications", "The 20 cities with the most adoption applications")

alt.LayerChart(...)

In [146]:
alt_pie_chart(city_counts[20:39], "City and State", "Applications", "21st-39th cities with the most adoption applications")

alt.LayerChart(...)

In [148]:
sum(city_counts[city_counts["Applications"] > 50]["Applications"])/sum(city_counts["Applications"])

0.6069350145489816

In [149]:
city_counts[city_counts["Applications"] > 45]

,City and State,Applications
0,"Haverhill, MA",590
1,"Amesbury, MA",477
2,"Newburyport, MA",380
3,"Salisbury, MA",253
4,"Boston, MA",175
5,"Hampton, NH",153
6,"Exeter, NH",145
7,"Seabrook, NH",135
8,"Merrimac, MA",135
9,"North Andover, MA",124


In [150]:
adopt_counts = adoptions["City and State"].value_counts().reset_index()
adopt_counts = adopt_counts.rename(columns={"count":"Adoptions"})

In [151]:
relevant_cities = adopt_counts[adopt_counts["City and State"].isin(city_counts[city_counts["Applications"] > 50]["City and State"].unique())].reset_index()

In [152]:
relevant_cities

,index,City and State,Adoptions
0,0,"Haverhill, MA",239
1,1,"Amesbury, MA",224
2,2,"Newburyport, MA",173
3,3,"Salisbury, MA",110
4,4,"Hampton, NH",67
5,5,"Exeter, NH",62
6,6,"Merrimac, MA",62
7,7,"Seabrook, NH",56
8,8,"North Andover, MA",52
9,9,"Ipswich, MA",51


In [153]:
alt_pie_chart(relevant_cities[:20], "City and State", "Adoptions", "Adoption counts (top 20 cities)")

alt.LayerChart(...)

In [156]:
alt_pie_chart(relevant_cities[20:], "City and State", "Adoptions", "Adoption counts (top 21st-39th cities)")

alt.LayerChart(...)

In [157]:
sum(relevant_cities["Adoptions"])/sum(adopt_counts["Adoptions"])

0.6778105539167486

## Q2: What adoption patterns can MRFRS capitalize on to more successfully adopt out cats with behavioral and medical issues?

## Q3: Are adoption events more effective than ”business as usual”?